In [2]:
from transformers import pipeline

In [94]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound, VideoUnavailable

In [5]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

model.safetensors:  48%|####7     | 776M/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [100]:
def get_video_summary(video_transcript):
  max = len(video_transcript.split(' '))//1.25
  min = len(video_transcript.split(' '))//2
  summary = summarizer(video_transcript, max_length = max, min_length = min)
  return summary[0]['summary_text']

In [101]:
def get_video_transcript(video_id):
  try:
      # Fetch transcript
      transcript_list = YouTubeTranscriptApi.get_transcript(video_id)

      # Combine transcript segments into a single text
      transcript_text = " ".join([item['text'] for item in transcript_list])
      return transcript_text

  except TranscriptsDisabled:
      return "Error: Transcripts are disabled for this video."

  except NoTranscriptFound:
      try:
          transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
          available_transcript = transcript_list.find_transcript(['en'])
          transcript = available_transcript.fetch()
          transcript_text = " ".join([item['text'] for item in transcript])
          return transcript_text
      except:
          return "Error: No transcript found for this video in any language."

  except VideoUnavailable:
      return "Error: The video is unavailable. It might be private or deleted."

  except IndexError:
      return "Error: Invalid YouTube URL format."

  except Exception as e:
      return f"Error: An unexpected error occurred: {str(e)}"

In [102]:
def get_video_id(video_link):
  video_id = video_link.split('v=')[1].split('&')[0]
  return video_id

In [111]:
def main(video_link):
  video_id = get_video_id(video_link)
  video_transcript = get_video_transcript(video_id)
  video_summary = get_video_summary(video_transcript)
  if video_transcript.split(' ')[0] == "Error:":
    return video_transcript
  else:
    return video_summary


In [ ]:
if __name__ == "__main__":
  video_link = str(input("Enter YouTube video link:"))
  print(main(video_link))